In [74]:
! pip install transformers datasets
from transformers import AutoTokenizer
import datasets
from typing import Any, Dict
import torch


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [85]:
ds = datasets.load_dataset('activated-ai/tiny-stories-8k-tokens', split='train')
tokenizer = AutoTokenizer.from_pretrained('activated-ai/tiny-stories-8k-tokenizer')


In [43]:
%%time
sum_chars = 0
for idx, doc in enumerate(ds['tokens']):
    sum_chars += len(tokenizer.decode(doc))
    if idx % 1000 == 0:
        print(idx, sum_chars)
print(idx, sum_chars, sum_chars / idx)

0 354
1000 797355
2000 1602938
3000 2505903
4000 3323266
5000 4089724
6000 4962688
7000 5835465
8000 6729630
9000 7605934
10000 8337121
11000 9256388
12000 10121035
13000 11012785
14000 11908268
15000 12762298
16000 13629564
17000 14723008
18000 15703973
19000 16634831
20000 17421447
21000 18423289
21989 19300244 877.7226795215789
CPU times: user 18.4 s, sys: 176 ms, total: 18.6 s
Wall time: 18.6 s


In [70]:
%%time

def count_chars(batch: Dict[str, Any], tokenizer):
    decoded_texts: List[str] = tokenizer.batch_decode(batch['tokens'], skip_special_tokens=True)    
    return {
        'char_count': [len(decoded_text) for decoded_text in decoded_texts]
        }
    pass


def parallel_count_chars(dataset, tokenizer, num_proc=18): 
    return dataset.map(
        lambda batch: count_chars(batch, tokenizer),
        batched=True,
        num_proc=num_proc,
        desc="Counting tokens"
    )

# Assuming 'ds' is your dataset and 'tok' is your FastTokenizer
result = parallel_count_chars(ds, tokenizer)

# Calculate total token count
total_chars, total_docs = sum(result['char_count']), len(result['char_count'])
print(f"Total number of chars {total_chars} in {total_docs} documents. Average chars per document: {total_chars / total_docs}")


Counting tokens (num_proc=18):   0%|          | 0/21990 [00:00<?, ? examples/s]

Total number of chars 19190294 in 21990 documents. Average chars per document: 872.6827648931333
CPU times: user 268 ms, sys: 205 ms, total: 474 ms
Wall time: 2.53 s


In [88]:
%%time
def process_batch(batch, tokenizer):
    decoded_texts: List[str] = tokenizer.batch_decode(batch['tokens'], skip_special_tokens=True)    
    
    # token_counts = [len(tokenizer.encode(text)) for text in decoded_texts]
    # flat_tokens = [token_list for token_list in batch['tokens']]
    return {
        # 'char_count': [len(decoded_text) for decoded_text in decoded_texts],
        'tokens': batch['tokens']
    }

def parallel_process(dataset, tokenizer, num_proc=32):
    return dataset.map(
        lambda batch: process_batch(batch, tokenizer),
        batched=True,
        num_proc=num_proc,
        remove_columns=dataset.column_names,
        desc="Processing tokens"
    )

# Assuming 'ds' is your dataset and 'tokenizer' is your FastTokenizer
# result = parallel_process(ds, tokenizer)

# Calculate total token count
# total_chars, num_docs = sum(result['char_count']), len(result['char_count'])
# print(f"Total number of tokens: {total_chars} in {num_docs} documents. Average tokens per document: {total_chars / num_docs}")

# Create a flat tensor
flattened_tokens = [token_id for token_list in result['tokens'] for token_id in token_list]
flat_tensor = torch.tensor(flattened_tokens, dtype=torch.int16)
print(f"Flat tensor shape: {flat_tensor.shape}")

Processing tokens (num_proc=32):   0%|          | 0/2119719 [00:00<?, ? examples/s]

KeyError: "Column char_count not in the dataset. Current columns in the dataset: ['tokens']"